In [1]:
import os, time
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

In [2]:
SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
SEED_BKG_DIR = '/data/darknet/python/background/'
GENE_IMG_DIR = '../generated/images/jpgs/'
GENE_XML_DIR = '../generated/xml/'
GENE_OBJ_DIR = '../generated/images/'

In [3]:
# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

IOU([1,2,3,4], [1,2,3,3.5])


0.5

In [4]:
def blur_background_from_image_file(file, bboxes, extend_spaces=0):
    
    img = cv2.imread(file)
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    img = cv2.blur(img, (int(img.shape[0]/20), int(img.shape[1]/30)))
    #img = cv2.blur(img, (32, 32))
    img = cv2.blur(img, (int(img.shape[0]/80), int(img.shape[1]/120)))
    #img = cv2.blur(img, (8, 8))
    i = 0
    for bbox in bboxes:
        img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], img)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

In [19]:
def past_to_background_from_image_file(file, bboxes, background_img_array, extend_spaces=0):
    
    img = cv2.imread(file)
    #img = img - 50
    if(img.shape != background_img_array.shape):
        print('shape not match')
        return
    #print(img.shape)
    #print(img)
    img_objs = []
    for bbox in bboxes:
        img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
        img_objs.append(img_obj)
    i = 0
    for bbox in bboxes:
        background_img_array[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])] = img_objs[i]
        i = i+1
    cv2.imwrite(GENE_IMG_DIR+file.split('/')[-1], background_img_array)
    return 
def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

In [20]:
seed_xml_names = os.listdir(SEED_XML_DIR)
seed_bkg_names = os.listdir(SEED_BKG_DIR)
bkg_heads = [bkg[:8] for bkg in seed_bkg_names]
print(bkg_heads)

seed_xml_names.sort()
#print(seed_xml_names)
#seed_img_names = os.listdir(SEED_IMG_DIR)
#seed_chg_names = os.listdir(SEED_CHG_DIR)

classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']
temp_index_array = [301, 302, 303]
#temp_index_array = [204]
last_frame_bboxes = []
for xml_name in seed_xml_names:
    #print(xml_name)       
    head = xml_name[:8]
    if(not (int(head) in temp_index_array)):
        continue
    index = bkg_heads.index(head)
    background_img_array = cv2.imread(SEED_BKG_DIR + seed_bkg_names[index])    
    '''
    head = xml_name[:11]
    if(not (int(head) > 214000 and int(head) < 215000)):
        continue
    '''
    if(not xml_name.endswith('.xml')):
        continue
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    
    #w = int(size.find('width').text)    
    #h = int(size.find('height').text)
    this_frame_bboxes = []
    boxes = []
    objs = []
    b = []
    #wrong_detection_objs = [[130, 311, 226, 475]]
    #objs_iter = root.iter('object')
    N = 0
    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        stable_bbox_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        
        '''
        for erro_bbox in wrong_detection_objs:
            if(IOU(erro_bbox, b) > 0.8):
                print(b)
                root.remove(obj)
                erro_obj_removed = True
                break
        if(erro_obj_removed):
            continue
        '''
        this_frame_bboxes.append(b)
        if(len(last_frame_bboxes)>0):
            for stable_bbox in last_frame_bboxes:
                if(IOU(stable_bbox, b) > 0.85):
                    #print(b)
                    root.remove(obj)
                    stable_bbox_removed = True
                    N = N+1
                    break
            if(stable_bbox_removed):
                continue
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 or bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue 
        else:
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))
        boxes.append(b)
        objs += obj
    last_frame_bboxes = this_frame_bboxes.copy()  
    #print(boxes)
    ###blur_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', boxes)
    if(len(boxes) == 0):
        continue
    past_to_background_from_image_file(SEED_IMG_DIR + xml_name[:-3] + 'jpg', boxes, background_img_array)
    tree.write(GENE_XML_DIR + xml_name)
    #break

['00000305', '00000300', '00000302', '00000304', '00000303', '00000301']
add:46.0:125.0
add:125.0:171.0
add:49.0:98.0
add:62.0:127.0
add:36.0:63.0
add:33.0:58.0
add:38.0:78.0
add:62.0:97.0
add:109.0:124.0
add:35.0:57.0
add:113.0:167.0
add:28.0:71.0
add:52.0:93.0
add:48.0:106.0
add:57.0:109.0
add:31.0:66.0
add:69.0:137.0
add:99.0:111.0
add:33.0:130.0
add:106.0:131.0
add:58.0:115.0
add:66.0:105.0
add:38.0:84.0
add:36.0:64.0
add:58.0:88.0
add:28.0:64.0
add:35.0:72.0
add:56.0:105.0
add:38.0:64.0
add:82.0:141.0
add:41.0:61.0
add:40.0:125.0
add:95.0:105.0
add:59.0:102.0
add:65.0:93.0
add:37.0:72.0
add:88.0:99.0
add:53.0:100.0
add:56.0:111.0
add:28.0:63.0
add:37.0:69.0
add:55.0:125.0
add:41.0:71.0
add:29.0:65.0
add:31.0:55.0
add:38.0:58.0
add:25.0:63.0
add:85.0:131.0
add:34.0:128.0
add:65.0:97.0
add:56.0:83.0
add:56.0:95.0
add:78.0:128.0
add:54.0:99.0
add:26.0:67.0
add:37.0:58.0
add:56.0:93.0
add:34.0:64.0
add:26.0:57.0
add:39.0:74.0
add:32.0:61.0
add:41.0:122.0
add:67.0:89.0
add:59.0:97.0
ad

add:27.0:88.0
add:106.0:32.0
add:66.0:111.0
add:25.0:60.0
add:24.0:62.0
add:39.0:77.0
add:30.0:89.0
add:66.0:117.0
add:103.0:30.0
add:23.0:60.0
add:32.0:62.0
add:28.0:96.0
add:26.0:61.0
add:62.0:106.0
add:106.0:29.0
add:25.0:58.0
add:25.0:58.0
add:33.0:63.0
add:61.0:95.0
add:107.0:29.0
add:25.0:52.0
add:29.0:100.0
add:27.0:70.0
add:62.0:97.0
add:25.0:65.0
add:108.0:28.0
add:27.0:89.0
add:32.0:73.0
add:62.0:101.0
add:22.0:70.0
add:17.0:34.0
add:40.0:94.0
add:21.0:71.0
add:25.0:75.0
add:10.0:38.0
add:112.0:25.0
add:26.0:109.0
add:32.0:79.0
add:82.0:159.0
add:41.0:132.0
add:21.0:41.0
add:119.0:24.0
add:67.0:128.0
add:32.0:107.0
add:103.0:124.0
add:33.0:72.0
add:30.0:42.0
add:53.0:98.0
add:109.0:22.0
add:31.0:111.0
add:57.0:133.0
add:97.0:115.0
add:38.0:47.0
add:34.0:56.0
add:30.0:105.0
add:94.0:113.0
add:30.0:92.0
add:68.0:129.0
add:48.0:42.0
add:110.0:22.0
add:35.0:73.0
add:28.0:114.0
add:56.0:135.0
add:29.0:89.0
add:81.0:113.0
add:55.0:40.0
add:39.0:74.0
add:34.0:96.0
add:34.0:106.0
add

add:77.0:182.0
add:52.0:179.0
add:28.0:94.0
add:34.0:58.0
add:25.0:64.0
add:44.0:94.0
add:68.0:214.0
add:57.0:187.0
add:28.0:87.0
add:27.0:69.0
add:20.0:82.0
add:42.0:70.0
add:71.0:207.0
add:68.0:179.0
add:23.0:69.0
add:20.0:97.0
add:30.0:106.0
add:47.0:139.0
add:42.0:78.0
add:23.0:105.0
add:22.0:68.0
add:64.0:211.0
add:67.0:218.0
add:25.0:95.0
add:24.0:95.0
add:47.0:144.0
add:39.0:86.0
add:106.0:156.0
add:58.0:74.0
add:30.0:66.0
add:64.0:202.0
add:62.0:235.0
add:35.0:143.0
add:51.0:148.0
add:26.0:107.0
add:35.0:92.0
add:30.0:86.0
add:66.0:92.0
add:108.0:164.0
add:24.0:108.0
add:46.0:110.0
add:25.0:54.0
add:91.0:219.0
add:57.0:231.0
add:37.0:121.0
add:28.0:69.0
add:70.0:93.0
add:26.0:126.0
add:34.0:79.0
add:26.0:100.0
add:25.0:71.0
add:35.0:149.0
add:72.0:212.0
add:30.0:85.0
add:53.0:211.0
add:24.0:61.0
add:40.0:143.0
add:47.0:108.0
add:27.0:46.0
add:54.0:151.0
add:65.0:187.0
add:83.0:230.0
add:30.0:88.0
add:20.0:87.0
add:22.0:59.0
add:28.0:121.0
add:59.0:167.0
add:31.0:90.0
add:75.0:1

add:41.0:133.0
add:37.0:159.0
add:92.0:91.0
add:26.0:94.0
add:58.0:114.0
add:24.0:84.0
add:25.0:77.0
add:36.0:67.0
add:143.0:63.0
add:26.0:82.0
add:33.0:140.0
add:33.0:140.0
add:54.0:98.0
add:46.0:138.0
add:24.0:69.0
add:140.0:61.0
add:26.0:78.0
add:38.0:74.0
add:26.0:77.0
add:47.0:138.0
add:46.0:185.0
add:30.0:77.0
add:22.0:65.0
add:51.0:96.0
add:129.0:82.0
add:91.0:98.0
add:17.0:75.0
add:45.0:145.0
add:25.0:122.0
add:60.0:134.0
add:43.0:176.0
add:139.0:89.0
add:56.0:19.0
add:47.0:121.0
add:56.0:108.0
add:25.0:67.0
add:25.0:71.0
add:22.0:95.0
add:16.0:20.0
add:31.0:63.0
add:102.0:119.0
add:51.0:145.0
add:38.0:181.0
add:151.0:102.0
add:25.0:123.0
add:61.0:132.0
add:23.0:58.0
add:10.0:20.0
add:38.0:170.0
add:69.0:147.0
add:152.0:105.0
add:56.0:137.0
add:24.0:123.0
add:23.0:117.0
add:22.0:89.0
add:50.0:113.0
add:21.0:90.0
add:90.0:97.0
add:14.0:30.0
add:53.0:107.0
add:53.0:175.0
add:56.0:142.0
add:153.0:100.0
add:110.0:38.0
add:26.0:107.0
add:15.0:31.0
add:13.0:24.0
add:23.0:127.0
add:54

add:66.0:149.0
add:37.0:82.0
add:96.0:78.0
add:20.0:58.0
add:47.0:121.0
add:56.0:139.0
add:23.0:72.0
add:102.0:87.0
add:44.0:151.0
add:29.0:109.0
add:26.0:88.0
add:52.0:96.0
add:27.0:79.0
add:71.0:64.0
add:28.0:57.0
add:58.0:104.0
add:37.0:122.0
add:31.0:77.0
add:31.0:69.0
add:28.0:67.0
add:49.0:88.0
add:26.0:63.0
add:39.0:76.0
add:38.0:124.0
add:36.0:79.0
add:56.0:103.0
add:26.0:63.0
add:49.0:88.0
add:36.0:78.0
add:28.0:80.0
add:68.0:65.0
add:33.0:69.0
add:31.0:76.0
add:52.0:100.0
add:47.0:133.0
add:30.0:106.0
add:30.0:113.0
add:39.0:122.0
add:51.0:114.0
add:33.0:70.0
add:31.0:87.0
add:38.0:123.0
add:52.0:105.0
add:50.0:121.0
add:116.0:129.0
add:107.0:105.0
add:32.0:67.0
add:71.0:81.0
add:49.0:89.0
add:34.0:69.0
add:33.0:110.0
add:42.0:123.0
add:36.0:71.0
add:119.0:148.0
add:25.0:72.0
add:34.0:65.0
add:116.0:140.0
add:23.0:86.0
add:17.0:96.0
add:32.0:76.0
add:36.0:81.0
add:109.0:128.0
add:50.0:119.0
add:30.0:86.0
add:20.0:75.0
add:72.0:106.0
add:128.0:172.0
add:114.0:167.0
add:42.0:12

...

add:25.0:67.0
add:66.0:105.0
add:57.0:145.0
add:27.0:75.0
add:26.0:81.0
add:48.0:119.0
add:28.0:68.0
add:21.0:78.0
add:45.0:116.0
add:64.0:150.0
add:101.0:115.0
add:34.0:82.0
add:31.0:58.0
add:65.0:110.0
add:69.0:94.0
add:23.0:84.0
add:51.0:111.0
add:32.0:73.0
add:33.0:89.0
add:82.0:139.0
add:38.0:71.0
add:35.0:85.0
add:58.0:122.0
add:58.0:131.0
add:46.0:108.0
add:39.0:62.0
add:35.0:80.0
add:30.0:76.0
add:30.0:88.0
add:73.0:154.0
add:43.0:108.0
add:30.0:98.0
add:32.0:55.0
add:44.0:107.0
add:27.0:89.0
add:103.0:126.0
add:25.0:57.0
add:27.0:78.0
add:77.0:171.0
add:79.0:157.0
add:34.0:148.0
add:43.0:101.0
add:29.0:75.0
add:40.0:112.0
add:34.0:90.0
add:80.0:115.0
add:28.0:67.0
add:53.0:108.0
add:62.0:133.0
add:26.0:79.0
add:28.0:106.0
add:47.0:95.0
add:132.0:124.0
add:80.0:151.0
add:82.0:151.0
add:29.0:72.0
add:36.0:93.0
add:42.0:79.0
add:58.0:125.0
add:43.0:116.0
add:38.0:52.0
add:102.0:107.0
add:26.0:65.0
add:72.0:147.0
add:59.0:124.0
add:78.0:167.0
add:46.0:137.0
add:39.0:141.0
add:80.0

add:59.0:150.0
add:64.0:125.0
add:38.0:124.0
add:28.0:58.0
add:39.0:79.0
add:36.0:89.0
add:28.0:54.0
add:41.0:62.0
add:18.0:74.0
add:55.0:49.0
add:47.0:42.0
add:48.0:48.0
add:61.0:140.0
add:37.0:118.0
add:66.0:122.0
add:58.0:51.0
add:14.0:46.0
add:29.0:56.0
add:90.0:94.0
add:35.0:73.0
add:52.0:145.0
add:32.0:50.0
add:28.0:119.0
add:64.0:135.0
add:56.0:134.0
add:61.0:113.0
add:50.0:96.0
add:51.0:44.0
add:64.0:69.0
add:27.0:53.0
add:42.0:76.0
add:104.0:138.0
add:30.0:18.0
add:64.0:51.0
add:15.0:49.0
add:29.0:49.0
add:35.0:77.0
add:55.0:109.0
add:31.0:116.0
add:40.0:82.0
add:74.0:132.0
add:54.0:51.0
add:101.0:115.0
add:12.0:44.0
add:56.0:107.0
add:36.0:64.0
add:27.0:56.0
add:33.0:52.0
add:44.0:18.0
add:20.0:64.0
add:27.0:54.0
add:28.0:101.0
add:58.0:119.0
add:54.0:104.0
add:62.0:88.0
add:59.0:54.0
add:49.0:98.0
add:93.0:107.0
add:31.0:57.0
add:64.0:17.0
add:54.0:106.0
add:17.0:45.0
add:40.0:142.0
add:63.0:130.0
add:19.0:39.0
add:38.0:95.0
add:55.0:47.0
add:31.0:59.0
add:79.0:116.0
add:84.

add:44.0:126.0
add:28.0:92.0
add:52.0:101.0
add:126.0:131.0
add:15.0:50.0
add:69.0:127.0
add:55.0:126.0
add:23.0:128.0
add:68.0:32.0
add:35.0:21.0
add:48.0:132.0
add:26.0:88.0
add:114.0:131.0
add:67.0:120.0
add:80.0:31.0
add:25.0:98.0
add:45.0:129.0
add:57.0:111.0
add:32.0:58.0
add:30.0:86.0
add:41.0:129.0
add:55.0:107.0
add:48.0:108.0
add:100.0:124.0
add:23.0:121.0
add:101.0:113.0
add:89.0:31.0
add:49.0:95.0
add:25.0:88.0
add:35.0:60.0
add:47.0:126.0
add:30.0:62.0
add:28.0:77.0
add:53.0:102.0
add:45.0:136.0
add:57.0:167.0
add:45.0:102.0
add:49.0:103.0
add:40.0:84.0
add:20.0:127.0
add:39.0:67.0
add:83.0:112.0
add:11.0:36.0
add:103.0:26.0
add:70.0:143.0
add:43.0:134.0
add:25.0:65.0
add:27.0:75.0
add:26.0:135.0
add:47.0:104.0
add:52.0:95.0
add:73.0:99.0
add:148.0:190.0
add:34.0:93.0
add:53.0:44.0
add:108.0:27.0
add:53.0:90.0
add:46.0:130.0
add:63.0:136.0
add:21.0:98.0
add:30.0:71.0
add:36.0:91.0
add:56.0:65.0
add:52.0:106.0
add:23.0:69.0
add:31.0:96.0
add:76.0:98.0
add:11.0:34.0
add:127.

add:47.0:131.0
add:52.0:165.0
add:30.0:79.0
add:60.0:132.0
add:23.0:38.0
add:49.0:101.0
add:39.0:126.0
add:39.0:168.0
add:59.0:130.0
add:31.0:85.0
add:23.0:38.0
add:46.0:132.0
add:48.0:165.0
add:50.0:123.0
add:45.0:127.0
add:62.0:170.0
add:15.0:30.0
add:46.0:131.0
add:47.0:172.0
add:46.0:118.0
add:33.0:78.0
add:19.0:21.0
add:35.0:61.0
add:44.0:137.0
add:44.0:174.0
add:44.0:105.0
add:32.0:70.0
add:52.0:168.0
add:47.0:144.0
add:41.0:119.0
add:47.0:143.0
add:27.0:63.0
add:44.0:130.0
add:36.0:149.0
add:38.0:89.0
add:41.0:131.0
add:27.0:84.0
add:30.0:68.0
add:51.0:161.0
add:39.0:146.0
add:30.0:82.0
add:40.0:84.0
add:31.0:69.0
add:32.0:83.0
add:61.0:156.0
add:32.0:88.0
add:33.0:78.0
add:38.0:67.0
add:50.0:164.0
add:35.0:92.0
add:36.0:81.0
add:47.0:171.0
add:31.0:90.0
add:32.0:64.0
add:56.0:174.0
add:33.0:82.0
add:31.0:60.0
add:54.0:165.0
add:32.0:89.0
add:60.0:116.0
add:58.0:167.0
add:33.0:94.0
add:71.0:170.0
add:30.0:93.0
add:34.0:92.0
add:45.0:173.0
add:52.0:126.0
add:60.0:168.0
add:48.0:1

add:53.0:89.0
add:83.0:37.0
add:58.0:105.0
add:39.0:82.0
add:118.0:186.0
add:38.0:121.0
add:42.0:76.0
add:54.0:196.0
add:50.0:97.0
add:28.0:97.0
add:29.0:118.0
add:62.0:107.0
add:57.0:94.0
add:73.0:115.0
add:58.0:93.0
add:61.0:118.0
add:78.0:129.0
add:83.0:38.0
add:106.0:127.0
add:81.0:38.0
add:59.0:227.0
add:39.0:81.0
add:147.0:184.0
add:93.0:245.0
add:134.0:167.0
add:70.0:208.0
add:59.0:167.0
add:131.0:166.0
add:118.0:170.0
add:38.0:123.0
add:59.0:107.0
add:57.0:94.0
add:77.0:150.0
add:67.0:160.0
add:111.0:130.0
add:59.0:95.0
add:64.0:163.0
add:103.0:128.0
add:36.0:60.0
add:20.0:92.0
add:109.0:143.0
add:67.0:174.0
add:77.0:131.0
add:41.0:73.0
add:36.0:67.0
add:64.0:143.0
add:31.0:116.0
add:124.0:133.0
add:60.0:178.0
add:100.0:122.0
add:80.0:161.0
add:60.0:171.0
add:98.0:120.0
add:161.0:147.0
add:55.0:118.0
add:58.0:90.0
add:103.0:124.0
add:73.0:168.0
add:142.0:159.0
add:92.0:130.0
add:55.0:144.0
add:55.0:102.0
add:144.0:169.0
add:104.0:145.0
add:67.0:180.0
add:90.0:131.0
add:56.0:167

In [13]:
def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

In [14]:
def generate_new_xmlobj(xmlobj_old, new_position, new_size):
    element_object = ET.Element('object')
    tag_name = ET.SubElement(element_object, 'name')
    tag_name.text = xmlobj_old.find('name').text

    tag_difficult = ET.SubElement(element_object, 'difficult')
    tag_difficult.text = xmlobj_old.find('difficult').text

    element_bndbox = ET.SubElement(element_object, 'bndbox')
    tag_xmin = ET.SubElement(element_bndbox, 'xmin')
    tag_ymin = ET.SubElement(element_bndbox, 'ymin')
    tag_xmax = ET.SubElement(element_bndbox, 'xmax')
    tag_ymax = ET.SubElement(element_bndbox, 'ymax')
    tag_xmin.text = str(new_position[0])
    tag_ymin.text = str(new_position[1])
    tag_xmax.text = str(new_position[0] + new_size[1])
    tag_ymax.text = str(new_position[1] + new_size[0])
    return element_object
    
def past_and_insert(img_obj, img_array, new_position, obj_element, etree):
    new_xmlobj = generate_new_xmlobj(obj_element, new_position, img_obj.shape[:2])
    new_xml_etree = insert_to_xml(new_xmlobj, etree)
    new_pil_img = past_obj_to_background(img_obj, img_array, new_position)
    return new_pil_img, new_xml_etree
    

def insert_to_xml(xml_obj, xml_etree):
    root = xml_etree.getroot()
    root.append(xml_obj)
    return xml_etree
    
def past_obj_to_background(img_obj, img_array, position = (200, 200)):
    img = Image.fromarray(img_array)
    img_obj = Image.fromarray(img_obj)
    img.paste(img_obj, position)
    return img
    
def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
def generate_new_position(img_size, img_obj_size):
    array_x = np.arange(int(img_size[1] - img_obj_size[1]))
    array_y = np.arange(int(img_size[0] - img_obj_size[0]))
    random_x = random.sample(list(array_x), 1)[0]
    random_y = random.sample(list(array_y), 1)[0]
    new_position = (random_x, random_y)
    #print(new_position)
    return new_position
    
    
def inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree):
    new_position = generate_new_position(img.shape[:2], img_obj.shape[:2])
    new_pil_img, new_xml_etree = past_and_insert(img_obj, img, new_position, obj_element, tree)
    
    return new_pil_img, new_xml_etree

In [15]:
gene_xml_names = os.listdir(GENE_XML_DIR)
for xml_name in gene_xml_names:
    '''
    if(not (int(xml_name[:-7]) == 204)):
        continue
    '''
    img_name = GENE_IMG_DIR + xml_name[:-3] + 'jpg'
    xml_name = GENE_XML_DIR + xml_name
    tree=ET.parse(xml_name)
    img = cv2.imread(img_name)
    
    objs = get_obj_from_xml(xml_name)
    i = 0
    for obj_element in objs:
        bbox = get_bbox_from_xmlobj(obj_element)           
        img_obj = get_obj_from_image_file(img_name, bbox)
        cls = get_cls_from_xmlobj(obj_element)
        #print(cls)
        cls_path = Path(GENE_OBJ_DIR + cls)
        if(not cls_path.is_dir()):
            os.system("mkdir " + GENE_OBJ_DIR + cls)
        save_path_name = GENE_OBJ_DIR + cls + "/" + xml_name.split('/')[-1][:-4] + "_" + str(i) + ".jpg"
        #print(save_path_name)
        cv2.imwrite(save_path_name, img_obj)
        i = i+1
        #new_pil_img, new_xml_etree = inset_obj_to_an_image_and_xml(img_obj, img, obj_element, tree)
        #tree = new_xml_etree
        #img = np.array(new_pil_img)

    ## save xml and img
    #new_pil_img.save('generated_' + img_name.split('/')[-1])
    #new_xml_etree.write('generated_' + xml_name.split('/')[-1])
    #break
   

In [ ]:
print(SEED_CHG_DIR)

In [36]:
def get_special_xml_according_to_special_images(dir_special_images, dir_xml):
    special_images = os.listdir(dir_special_images)
    #print(special_images)
    for special_image in special_images:
        special_xml = dir_xml + special_image[:-len('jpg')] + 'xml'
        #print(special_xml)
        os.system('cp ' + special_xml + ' ' + dir_special_images[:-(len('images') + 1)] + 'xml')

In [37]:
dir_special_images = '../generated/special_images/'
dir_xml = '../generated/xml/'
get_special_xml_according_to_special_images(dir_special_images, dir_xml)

(False, [])